**Пример работы программы**

Для начала инициализируем наши константы

In [2]:
import numpy as np
from SCL import SCL
from copy import deepcopy

N = 16 # Количество бит в готовом векторе
maxSize = 4 # Количество сохранённых путей
F = [0, 1, 2, 3, 4, 5, 8, 9] # Позиции замороженных бит
sigma = 0.3

Инициализируем вектор, наложим шум (и выведем итоговые значения с нумерацией, чтобы потом было легче добавлять ошибки)

In [3]:
scl = SCL(N, F, maxSize)
vector = [1]*8
vec_with_frozen_bits = scl.prepare_input(vector)
encoded_vec = scl.encode(vec_with_frozen_bits)
display(f'Закодированный вектор: {encoded_vec}')

np.random.seed(17)
awgn_encoded_vec = encoded_vec + sigma * np.random.randn(N)
print(f'Зашумлённый вектор: {awgn_encoded_vec}')
for i in range(N):
    print(i, awgn_encoded_vec[i], 'frozen' if i in F else 'info')

'Закодированный вектор: [ 1  1  1 -1  1 -1  1  1  1 -1  1  1  1  1  1 -1]'

Зашумлённый вектор: [ 1.08287977  0.44361158  1.18717033 -0.65640661  1.31115714 -0.43400832
  0.96649051  0.8913696   1.04460251 -1.13133495  1.6513771   1.34569308
  0.4543563   0.9585852   1.16195188 -1.53258469]
0 1.0828797670063957 frozen
1 0.4436115763580485 frozen
2 1.187170333397907 frozen
3 -0.656406613128373 frozen
4 1.3111571404683648 frozen
5 -0.4340083210655177 frozen
6 0.9664905129613701 info
7 0.8913695984802854 info
8 1.0446025135135732 frozen
9 -1.1313349457543898 frozen
10 1.651377099251993 info
11 1.3456930751870964 info
12 0.45435629820915135 info
13 0.9585851977451075 info
14 1.1619518841573024 info
15 -1.5325846861324894 info


Для начала убедимся, что без ошибок наш код раскодируется правильно

In [4]:
res = scl.decode(awgn_encoded_vec)
scl.print_decoded_results_table(*res)

метрика пути       | код с замороженными битами | код     
----------------------------------------------------------
0                  | 0000001100111111           | 11111111
2.4253961090322247 | 0000000000111111           | 00111111
2.9675400409230637 | 0000001100110011           | 11110011
3.3946932071260947 | 0000001100000011           | 11000011


Действительно, правильный вариант даже имеет метрику пути в 0

Случай с двумя ошибками (одна на замороженной позиции, другая на информационной)

In [5]:
vec_2_mistakes = deepcopy(awgn_encoded_vec)
vec_2_mistakes[1] -= 1
vec_2_mistakes[7] -= 1

res_2_mistakes = scl.decode(vec_2_mistakes)
scl.print_decoded_results_table(*res_2_mistakes)

метрика пути       | код с замороженными битами | код     
----------------------------------------------------------
0.6650188251616661 | 0000001100111111           | 11111111
1.0904149341938907 | 0000000000111111           | 00111111
2.304278272932204  | 0000000000000011           | 00000011
2.6325588660847297 | 0000001100110011           | 11110011


Верный код снова имеет лучшую метрику пути!

Рассмотрим теперь случай с тремя ошибками, и чтобы уменьшить эффект снежного кома, добавим ошибки на более поздние позиции

In [6]:
vec_3_mistakes = deepcopy(awgn_encoded_vec)
vec_3_mistakes[6] -= 1.2
vec_3_mistakes[9] += 1.3
vec_3_mistakes[12] -= 1

res_3_mistakes = scl.decode(vec_3_mistakes)
scl.print_decoded_results_table(*res_3_mistakes)

метрика пути       | код с замороженными битами | код     
----------------------------------------------------------
0.9478182430750888 | 0000001100111111           | 11111111
2.4004379011570904 | 0000000100110111           | 01110111
2.615358283998152  | 0000001100110011           | 11110011
2.6417740788712774 | 0000001100011101           | 11011101


Снова лучший показатель! На 4 ошибках затронем информационные биты пораньше, чтобы увеличить эффект от ошибок

In [7]:
vec_4_mistakes = deepcopy(awgn_encoded_vec)
vec_4_mistakes[1] -= 1.5
vec_4_mistakes[6] -= 1.5
vec_4_mistakes[7] -= 1.5
vec_4_mistakes[10] -= 2

res_4_mistakes = scl.decode(vec_4_mistakes)
scl.print_decoded_results_table(*res_4_mistakes)

метрика пути       | код с замороженными битами | код     
----------------------------------------------------------
1.9725473219805276 | 0000000000111111           | 00111111
2.547151212948303  | 0000001100111111           | 11111111
2.627650999409438  | 0000001000010111           | 10010111
3.186410660718841  | 0000000000000011           | 00000011


Правильный код оказался вторым в подборке! С учётом того, что ошибок мы накидали много и достаточно сильных, это всё ещё очень крутой результат